In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from mnist_cnn import MNISTCNN_Complex, MNIST

In [2]:
device = torch.device('mps' if torch.backends.mps.is_available() else 'cpu')
device

device(type='mps')

In [3]:
trainset, _, testset = MNIST()

trainloader = DataLoader(trainset, batch_size=64, shuffle=True)
testloader = DataLoader(testset, batch_size=1000, shuffle=False)

In [4]:
model = MNISTCNN_Complex().to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

for epoch in range(10): 
    model.train()
    running_loss = 0.0
    for inputs, labels in trainloader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch {epoch+1} - Loss: {running_loss / len(trainloader):.4f}")

Epoch 1 - Loss: 0.1972
Epoch 2 - Loss: 0.0542
Epoch 3 - Loss: 0.0401
Epoch 4 - Loss: 0.0342
Epoch 5 - Loss: 0.0271
Epoch 6 - Loss: 0.0253
Epoch 7 - Loss: 0.0205
Epoch 8 - Loss: 0.0199
Epoch 9 - Loss: 0.0187
Epoch 10 - Loss: 0.0140


In [5]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in testloader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total:.2f}%")

Test Accuracy: 98.94%


In [6]:
torch.save(model, './MNIST_CNN_Complex.pt')